In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%pip install ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 71.1 MB/s eta 0:00:00


In [3]:
from pathlib import Path

ROOT = Path("/content/drive/MyDrive/pothole_project")

TEST_ROOT = Path("/content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test")

MODEL_PATH = Path("/content/drive/MyDrive/pothole_project/data_raw/bbox_data/runs/weights/best.pt")

print("ROOT:", ROOT)
print("TEST_ROOT:", TEST_ROOT)
print("MODEL_PATH:", MODEL_PATH)
print("Model exists:", MODEL_PATH.exists())

images_dir = TEST_ROOT / "images"
labels_dir = TEST_ROOT / "labels"

print("Images dir:", images_dir, "exists:", images_dir.exists())
print("Labels dir:", labels_dir, "exists:", labels_dir.exists())

num_images = len(list(images_dir.glob("*.jpg")))
num_labels = len(list(labels_dir.glob("*.txt")))
print("Num images:", num_images)
print("Num labels:", num_labels)


ROOT: /content/drive/MyDrive/pothole_project
TEST_ROOT: /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test
MODEL_PATH: /content/drive/MyDrive/pothole_project/data_raw/bbox_data/runs/weights/best.pt
Model exists: True
Images dir: /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test/images exists: True
Labels dir: /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test/labels exists: True
Num images: 97
Num labels: 97


In [ ]:
import yaml

data_cfg = {
    "path": str(TEST_ROOT),   # root of the test dataset
    "train": "images",
    "val":   "images",
    "test":  "images",
    "nc": 1,
    "names": ["pothole"],
}

yaml_path = "/content/pothole_test_mwpd.yaml"
with open(yaml_path, "w") as f:
    yaml.safe_dump(data_cfg, f)

print("Wrote dataset yaml to:", yaml_path)
print("---- dataset.yaml contents ----")
print(open(yaml_path).read())


Wrote dataset yaml to: /content/pothole_test_mwpd.yaml
---- dataset.yaml contents ----
names:
- pothole
nc: 1
path: /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test
test: images
train: images
val: images



In [ ]:
from ultralytics import YOLO

model = YOLO(str(MODEL_PATH))

# Run eval
metrics = model.val(
    data=yaml_path,
    split="val",
    imgsz=640,
    batch=16
)

print("Metrics dict:")
print(metrics.results_dict)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.235 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (AMD EPYC 7B12)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs
val: Fast image access ✅ (ping: 0.9±0.3 ms, read: 0.3±0.2 MB/s, size: 69.8 KB)
val: Scanning /content/drive/.shortcut-targets-by-id/1YIu4lT2GJsDKU5pKMODIm2xqHEh8Xyid/pothole_project/data_raw/bbox_data/test_datasets/test/labels... 97 images, 0 backgrounds, 0 corrupt: 100% ━━━━━━━━━━━━ 97/97 1.9it/s 50.6s
val: New cache created: /content/drive/.shortcut-targets-by-id/1YIu4lT2GJsDKU5pKMODIm2xqHEh8Xyid/pothole_project/data_raw/bbox_data/test_datasets/test/labels.cache
                 Class     Images  Instances      Box(P          

In [ ]:
pred_out = "/content/yolo_mwpd_test_preds"

results = model.predict(
    source=str(TEST_ROOT / "images"),
    imgsz=640,
    conf=0.25,
    save=True,
    project=pred_out,
    name="run1"
)

print("Predictions saved under:", pred_out)
!find {pred_out} -maxdepth 3 -type f | head



image 1/97 /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test/images/12_jpg.rf.98223fbf0c9935d763f81b9e25e9712b.jpg: 640x640 2 potholes, 200.5ms
image 2/97 /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test/images/139_jpg.rf.f2928d6451a5fd5ef95a3051b4b2ba5a.jpg: 640x640 17 potholes, 153.6ms
image 3/97 /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test/images/147_jpg.rf.29a0769290af2d4a69db84c5b838eae7.jpg: 640x640 28 potholes, 141.9ms
image 4/97 /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test/images/15_jpg.rf.efee37dd5a47841089fa6fa96bcded28.jpg: 640x640 1 pothole, 141.7ms
image 5/97 /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test/images/181_jpg.rf.f5d2a9174fcd2f2b3aa547325f9863b0.jpg: 640x640 3 potholes, 164.8ms
image 6/97 /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test/images/19_jpg.rf.c0972876ad058389d3ec2943deae367d.jp

In [ ]:
!mkdir -p /content/drive/MyDrive/pothole_project/runs
!cp -r {pred_out} /content/drive/MyDrive/pothole_project/runs/yolo_mwpd_test_preds
print("Copied predictions to Drive:/pothole_project/runs/yolo_mwpd_test_preds")


Copied predictions to Drive:/pothole_project/runs/yolo_mwpd_test_preds


#Mask R-CNN detector

In [4]:
import torch
from pathlib import Path
from PIL import Image
import torchvision
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms.functional as F
import math
from typing import List, Dict

# Paths
TEST_ROOT = Path("/content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test")

SEG_MODEL_PATH = Path("/content/drive/MyDrive/pothole_project/pothole_seg/models/maskrcnn_pothole.pth")

print("Test root:", TEST_ROOT)
print("Segmentation model path:", SEG_MODEL_PATH, "exists:", SEG_MODEL_PATH.exists())

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)


Test root: /content/drive/MyDrive/pothole_project/data_raw/bbox_data/test_datasets/test
Segmentation model path: /content/drive/MyDrive/pothole_project/pothole_seg/models/maskrcnn_pothole.pth exists: True
Using device: cuda


In [5]:
import torchvision

num_classes = 2

# create a base Mask R-CNN with correct num_classes
model_seg = torchvision.models.detection.maskrcnn_resnet50_fpn(
    weights=None,
    num_classes=num_classes
)


state_dict = torch.load(SEG_MODEL_PATH, map_location="cpu")
model_seg.load_state_dict(state_dict)

model_seg.to(device)
model_seg.eval()

print("Loaded Mask R-CNN model.")


Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 184MB/s]


Loaded Mask R-CNN model.


In [6]:
class PotholeTestDataset(Dataset):
    def __init__(self, root: Path):
        self.root = root
        self.img_dir = root / "images"
        self.lbl_dir = root / "labels"
        self.images = sorted(list(self.img_dir.glob("*.jpg")))
        assert len(self.images) > 0, "No images found in images/"

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx: int):
        img_path = self.images[idx]
        label_path = self.lbl_dir / (img_path.stem + ".txt")


        img = Image.open(img_path).convert("RGB")
        w, h = img.size

        # Load YOLO labels
        boxes = []
        labels = []
        if label_path.exists():
            with open(label_path, "r") as f:
                for line in f:
                    parts = line.strip().split()
                    if len(parts) != 5:
                        continue
                    cls, cx, cy, bw, bh = parts
                    cls = int(cls)
                    cx = float(cx) * w
                    cy = float(cy) * h
                    bw = float(bw) * w
                    bh = float(bh) * h

                    x1 = cx - bw / 2.0
                    y1 = cy - bh / 2.0
                    x2 = cx + bw / 2.0
                    y2 = cy + bh / 2.0

                    boxes.append([x1, y1, x2, y2])
                    # all potholes → label 1
                    labels.append(1)

        if len(boxes) == 0:
            boxes = torch.zeros((0, 4), dtype=torch.float32)
            labels = torch.zeros((0,), dtype=torch.int64)
        else:
            boxes = torch.tensor(boxes, dtype=torch.float32)
            labels = torch.tensor(labels, dtype=torch.int64)

        # convert image to tensor
        img_t = F.to_tensor(img)

        target = {
            "boxes": boxes,
            "labels": labels,
            "image_id": torch.tensor([idx]),
        }

        return img_t, target

dataset_test = PotholeTestDataset(TEST_ROOT)
len(dataset_test), dataset_test[0][0].shape, dataset_test[0][1]


(97,
 torch.Size([3, 640, 640]),
 {'boxes': tensor([[ 22.7500, 333.0000, 538.2500, 613.0000]]),
  'labels': tensor([1]),
  'image_id': tensor([0])})

In [7]:
from torch.utils.data import DataLoader

def box_iou(boxes1: torch.Tensor, boxes2: torch.Tensor) -> torch.Tensor:
    """
    Compute IoU between two sets of boxes (x1,y1,x2,y2).
    """
    if boxes1.numel() == 0 or boxes2.numel() == 0:
        return torch.zeros((boxes1.shape[0], boxes2.shape[0]))
    area1 = (boxes1[:, 2] - boxes1[:, 0]).clamp(min=0) * (boxes1[:, 3] - boxes1[:, 1]).clamp(min=0)
    area2 = (boxes2[:, 2] - boxes2[:, 0]).clamp(min=0) * (boxes2[:, 3] - boxes2[:, 1]).clamp(min=0)

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]

    wh = (rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]

    union = area1[:, None] + area2 - inter
    iou = inter / (union + 1e-6)
    return iou

def evaluate_maskrcnn(model, dataset, iou_thresh=0.5, score_thresh=0.5, batch_size=2):
    model.eval()
    loader = DataLoader(dataset, batch_size=batch_size, shuffle=False, collate_fn=lambda x: list(zip(*x)))

    all_scores = []
    all_matches = []
    n_gt_total = 0

    with torch.no_grad():
        for imgs, targets in loader:
            imgs = [img.to(device) for img in imgs]
            outputs = model(imgs)

            for output, target in zip(outputs, targets):
                gt_boxes = target["boxes"].to(device)
                n_gt_total += gt_boxes.shape[0]

                if gt_boxes.numel() == 0 and output["boxes"].shape[0] == 0:
                    continue

                # predicted boxes and scores for class 1 (pothole)
                boxes = output["boxes"]
                scores = output["scores"]
                labels = output["labels"]

                keep = (scores >= score_thresh) & (labels == 1)
                boxes = boxes[keep]
                scores = scores[keep]

                if boxes.numel() == 0:
                    continue


                ious = box_iou(boxes, gt_boxes)

                # Greedy matching
                matched_gt = set()
                for i in range(boxes.shape[0]):
                    max_iou, j = (ious[i].max().item(), ious[i].argmax().item()) if gt_boxes.shape[0] > 0 else (0.0, -1)
                    if max_iou >= iou_thresh and j not in matched_gt:
                        all_matches.append(1)  # TP
                        matched_gt.add(j)
                    else:
                        all_matches.append(0)  # FP
                    all_scores.append(scores[i].item())

    if len(all_scores) == 0:
        print("No predictions above threshold; returning zeros.")
        return {
            "precision": 0.0,
            "recall": 0.0,
            "AP50": 0.0,
            "n_gt": n_gt_total,
        }

    # sort by confidence descending
    import numpy as np
    all_scores = np.array(all_scores)
    all_matches = np.array(all_matches)
    order = np.argsort(-all_scores)
    all_scores = all_scores[order]
    all_matches = all_matches[order]

    tp = all_matches
    fp = 1 - all_matches

    tp_cum = np.cumsum(tp)
    fp_cum = np.cumsum(fp)

    recalls = tp_cum / (n_gt_total + 1e-6)
    precisions = tp_cum / (tp_cum + fp_cum + 1e-6)


    def compute_ap(recalls, precisions):
        # ensure sorted by recall
        mrec = np.concatenate(([0.0], recalls, [1.0]))
        mpre = np.concatenate(([0.0], precisions, [0.0]))

        for i in range(mpre.size - 1, 0, -1):
            mpre[i - 1] = max(mpre[i - 1], mpre[i])
        # find points where recall changes
        idx = np.where(mrec[1:] != mrec[:-1])[0]
        ap = np.sum((mrec[idx + 1] - mrec[idx]) * mpre[idx + 1])
        return ap

    ap50 = compute_ap(recalls, precisions)
    precision_final = precisions[-1]
    recall_final = recalls[-1]

    return {
        "precision": float(precision_final),
        "recall": float(recall_final),
        "AP50": float(ap50),
        "n_gt": int(n_gt_total),
    }


In [8]:
metrics_seg = evaluate_maskrcnn(
    model_seg,
    dataset_test,
    iou_thresh=0.5,
    score_thresh=0.5,
    batch_size=2
)

print("Mask R-CNN evaluation on test dataset:")
print(metrics_seg)


Mask R-CNN evaluation on test dataset:
{'precision': 0.548275860178359, 'recall': 0.5445205460804091, 'AP50': 0.47471127818466113, 'n_gt': 292}


In [9]:
{
  'precision': ...,
  'recall': ...,
  'AP50': ...,
  'n_gt': 292
}


{'precision': Ellipsis, 'recall': Ellipsis, 'AP50': Ellipsis, 'n_gt': 292}